In [1]:
import jupyter_black
from dotenv import load_dotenv
from garminconnect import Garmin
import os
import datetime
from typing import Any
import numpy as np
from pydantic import BaseModel, Field
from openai import OpenAI
from enum import Enum
import json
from typing import Literal

jupyter_black.load()
load_dotenv(".envrc", override=True)

True

# GET GARMIN DETAILS

In [2]:
GARMIN_EMAIL = os.getenv("GARMIN_EMAIL")
GARMIN_PASSWORD = os.getenv("GARMIN_PASSWORD")
GARMIN_TOKENSTORE = os.getenv("GARMIN_TOKENSTORE")

In [3]:
print(GARMIN_TOKENSTORE)

~/.garminconnect_vittoria


In [4]:
def start_garmin() -> Garmin:
    """Initialize Garmin connection."""
    try:
        GARMIN = Garmin(
            email=GARMIN_EMAIL,
            password=GARMIN_PASSWORD,
            is_cn=False,  # if you are in China, set to True
        )
        GARMIN.login()
        print("You are now logged in.")
        return GARMIN
    except Exception as e:
        print(f"Could not login with email and password: {e}")
        raise

In [5]:
garmin = start_garmin()

You are now logged in.


# GET SUMMARY OF HEALTH DATA FOR AN INTERVAL OF DAYS

In [6]:
def get_daily_health_summary(
    api: Any, start: datetime.date, end: datetime.date
) -> list[dict[str, Any]]:
    """
    One dict per day using get_user_summary() where possible.
    Fields: resting_heart_rate, exercise_minutes, stress_level, sleep_hours, sleep_score, steps, body_battery_final
    """

    def dstr(d: datetime.date) -> str:
        return d.isoformat()

    def daterange(a: datetime.date, b: datetime.date):
        for i in range((b - a).days + 1):
            yield a + datetime.timedelta(days=i)

    out: list[dict[str, Any]] = []

    for day in daterange(start, end):
        s = dstr(day)
        day_of_week = day.strftime("%A")
        summary = api.get_user_summary(s) or {}
        rhr = summary.get("restingHeartRate")
        steps = summary.get("totalSteps")
        stress_level = summary.get("averageStressLevel")
        body_battery_final = summary.get("bodyBatteryMostRecentValue") or summary.get(
            "mostRecentBodyBattery"
        )
        exercise_minutes = (summary.get("moderateIntensityMinutes") or 0) + (
            summary.get("vigorousIntensityMinutes") or 0
        )
        sleep_seconds = summary.get("sleepingSeconds")
        sleep_hours = round(sleep_seconds / 3600, 2)
        body_battery_start = summary.get("bodyBatteryAtWakeTime")
        total_distance_meters = summary.get("totalDistanceMeters")

        out.append(
            {
                "date": s,
                "day_of_week": day_of_week,
                "resting_heart_rate": rhr,
                "exercise_minutes": exercise_minutes,
                "stress_level": stress_level,
                "sleep_hours": sleep_hours,
                "steps": steps,
                "total_distance_meters": total_distance_meters,
                "body_battery_start_day": body_battery_start,
                "body_battery_end_day": body_battery_final,
            }
        )

    return out

In [7]:
yesterday = datetime.date.today() - datetime.timedelta(days=1)
summary_for_yesterday = get_daily_health_summary(garmin, yesterday, yesterday)
summary_for_yesterday

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [ ]:
yesterday = datetime.date.today() - datetime.timedelta(days=1)
past_7_days_start = yesterday - datetime.timedelta(days=7)
past_7_days_end = yesterday - datetime.timedelta(days=1)
summary_for_past_7_days = get_daily_health_summary(
    garmin, past_7_days_start, past_7_days_end
)
summary_for_past_7_days

[{'date': '2025-08-11',
  'day_of_week': 'Monday',
  'resting_heart_rate': 43,
  'exercise_minutes': 39,
  'stress_level': 25,
  'sleep_hours': 7.46,
  'steps': 11111,
  'total_distance_meters': 11202,
  'body_battery_start_day': 76,
  'body_battery_end_day': 18},
 {'date': '2025-08-12',
  'day_of_week': 'Tuesday',
  'resting_heart_rate': 43,
  'exercise_minutes': 49,
  'stress_level': 30,
  'sleep_hours': 8.54,
  'steps': 13162,
  'total_distance_meters': 12712,
  'body_battery_start_day': 69,
  'body_battery_end_day': 14},
 {'date': '2025-08-13',
  'day_of_week': 'Wednesday',
  'resting_heart_rate': 44,
  'exercise_minutes': 0,
  'stress_level': 27,
  'sleep_hours': 8.61,
  'steps': 5571,
  'total_distance_meters': 4743,
  'body_battery_start_day': 70,
  'body_battery_end_day': 27},
 {'date': '2025-08-14',
  'day_of_week': 'Thursday',
  'resting_heart_rate': 44,
  'exercise_minutes': 39,
  'stress_level': 22,
  'sleep_hours': 7.93,
  'steps': 9637,
  'total_distance_meters': 9933,
  

# BUILD TRENDS AND PROMPT FOR DEEPSEEK

In [ ]:
def detect_trend(values, pct_threshold=5):
    recent, earlier = np.mean(values[-3:]), np.mean(values[:3])
    return (
        "up"
        if recent > earlier * (1 + pct_threshold / 100)
        else ("down" if recent < earlier * (1 - pct_threshold / 100) else "flat")
    )


def build_llm_context_md(
    summary_for_today: list[dict], summary_for_past_7_days: list[dict]
) -> str:
    """
    Creates a Markdown-formatted context string for an LLM.
    Clarifies:
    - The "7-day average" excludes today's value
    - The trend is based on the 7 days before today
    - Percent change is relative to that 7-day average
    """
    assert len(summary_for_today) == 1, "Expected 1 day of summary"
    today = summary_for_today[0]

    metrics = [key for key in today.keys() if key not in ["date", "day_of_week"]]
    lines = [
        f"# Daily Metrics Summary for {today['date']} ({today['day_of_week']})",
        "_Note: All comparisons use the **previous 7 days only**, excluding today._",
        "",
    ]

    better_is_lower = [
        "resting_heart_rate",
        "stress_level",
    ]

    for metric in metrics:
        today_val = today[metric]
        past_vals = [
            day[metric] for day in summary_for_past_7_days if day[metric] is not None
        ]
        avg_7d = sum(past_vals) / len(past_vals) if past_vals else 0
        delta_pct = ((today_val - avg_7d) / avg_7d * 100) if avg_7d else 0
        trend_dir = detect_trend(past_vals)
        arrow = "↑" if trend_dir == "up" else ("↓" if trend_dir == "down" else "→")

        lines.append(
            f"## {metric.replace('_', ' ').title()}\n"
            f"- Today's value ({today['date']}): {today_val}\n"
            f"- 7-day baseline average (excluding today): {avg_7d:.2f}\n"
            f"- Percent change vs. baseline: {delta_pct:+.1f}%\n"
            f"- Trend over previous 7 days: {trend_dir} {arrow}\n"
            f"- Better is lower: {metric in better_is_lower}\n"
        )

    return "\n".join(lines)


prompt = build_llm_context_md(summary_for_yesterday, summary_for_past_7_days)

print(prompt)

# Daily Metrics Summary for 2025-08-18 (Monday)
_Note: All comparisons use the **previous 7 days only**, excluding today._

## Resting Heart Rate
- Today's value (2025-08-18): 42
- 7-day baseline average (excluding today): 43.14
- Percent change vs. baseline: -2.6%
- Trend over previous 7 days: flat →
- Better is lower: True

## Exercise Minutes
- Today's value (2025-08-18): 0
- 7-day baseline average (excluding today): 42.57
- Percent change vs. baseline: -100.0%
- Trend over previous 7 days: up ↑
- Better is lower: False

## Stress Level
- Today's value (2025-08-18): 28
- 7-day baseline average (excluding today): 28.71
- Percent change vs. baseline: -2.5%
- Trend over previous 7 days: up ↑
- Better is lower: True

## Sleep Hours
- Today's value (2025-08-18): 7.98
- 7-day baseline average (excluding today): 6.50
- Percent change vs. baseline: +22.8%
- Trend over previous 7 days: down ↓
- Better is lower: False

## Steps
- Today's value (2025-08-18): 4867
- 7-day baseline average (excl

# DEFINE OUR STRUCTURED OUTPUT

In [ ]:
class DayType(str, Enum):
    TRAINING = "training"
    ACTIVE_RECOVERY = "active_recovery"
    REST = "rest"
    HIGH_STRESS = "high_stress"
    BALANCED = "balanced"


class DailySummary(BaseModel):
    day_type: DayType = Field(
        ...,
        description="Classification of the day based on activity and recovery metrics",
    )

    title: str = Field(..., description="One sentence summary of the day")
    emoji: str = Field(..., description="Emoji to represent the day type")
    observation: str = Field(
        ...,
        description="Two sentence observation about key metrics and patterns",
    )
    recommendation: str = Field(
        ...,
        description="Two sentence actionable recommendation for tomorrow",
    )

In [ ]:
training_day_example = DailySummary(
    day_type=DayType.TRAINING,
    title="Strong training day with elevated activity across all metrics.",
    emoji="💪",
    observation="Exercise minutes doubled your baseline with 95 minutes of activity, supported by 18,500 steps. Despite the high training load, body battery started at a solid 85, indicating good recovery from yesterday.",
    recommendation="Consider an active recovery or rest day tomorrow to allow adaptation from today's effort. Prioritize sleep tonight to maintain your body battery levels and support muscle recovery.",
)

# Example 2: High Stress Day
high_stress_day_example = DailySummary(
    day_type=DayType.HIGH_STRESS,
    emoji="😫",
    title="Elevated stress and poor recovery despite minimal physical activity.",
    observation="Stress levels jumped 46% above baseline while sleep dropped to just 6.1 hours, resulting in a low body battery start of 45. Exercise and movement were minimal, suggesting stress is from non-physical sources.",
    recommendation="Focus on stress management techniques and aim for 8+ hours of sleep tonight. Consider light exercise like walking or yoga tomorrow, as gentle movement can help regulate stress levels.",
)
examples = [training_day_example, high_stress_day_example]
examples_str = "\n\n\n".join([example.model_dump_json() for example in examples])

In [ ]:
system_prompt = f"""
Instructions: 
* You will be given a summary of the user's health and fitness data for today, in comparison to the past 7 days.
* Your goal is to generate a summary that will be shown in the user's smart watch. 
* Keep things short, but also interesting to the user. 
* Your summary should include a type of day, a title, some observations and recommendations for the user. 
* Your summary should be in JSON format. Only output the JSON, no other text.

---JSON SCHEMA---
{DailySummary.model_json_schema()}
---END JSON SCHEMA---

---EXAMPLE JSON OUTPUTS---
{examples_str}
---END EXAMPLE JSON OUTPUTS---
"""

In [ ]:
def llm(
    messages: list[dict], model: str, response_format: dict | None = None
) -> tuple[dict, str | None]:
    client = OpenAI(
        api_key=os.environ["DEEPSEEK_API_KEY"],
        base_url="https://api.deepseek.com",
    )
    response = client.chat.completions.create(
        model=model, messages=messages, response_format=response_format, temperature=0.0
    )
    message = response.choices[0].message

    if hasattr(message, "reasoning_content"):
        reasoning_content = message.reasoning_content
    else:
        reasoning_content = None

    return json.loads(message.content), reasoning_content

In [ ]:
# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": prompt},
# ]


# response, reasoning = llm(messages, "deepseek-chat", {"type": "json_object"})
# health_summary = DailySummary.model_validate(response)
# print("=" * 10, "Reasoning", "=" * 10)
# print(reasoning)
# print("=" * 10, "Health Summary", "=" * 10)
# print(health_summary)

In [ ]:
# response, reasoning = llm(messages, "deepseek-reasoner", {"type": "json_object"})
# health_summary = DailySummary.model_validate(response)
# print("=" * 10, "Reasoning", "=" * 10)
# print(reasoning)
# print("=" * 10, "Health Summary", "=" * 10)
# print(health_summary)

In [ ]:
def get_daily_summary(
    garmin: Garmin,
    date: str,
    model: Literal["deepseek-chat", "deepseek-reasoner"],
    verbose: bool = False,
) -> DailySummary:
    date_for_summary = datetime.datetime.strptime(date, "%Y-%m-%d").date()
    summary_in_date = get_daily_health_summary(
        garmin, date_for_summary, date_for_summary
    )

    past_period_start = date_for_summary - datetime.timedelta(days=7)
    past_period_end = date_for_summary - datetime.timedelta(days=1)
    summary_in_past_period = get_daily_health_summary(
        garmin, past_period_start, past_period_end
    )

    prompt = build_llm_context_md(summary_in_date, summary_in_past_period)

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    response, reasoning = llm(messages, model, {"type": "json_object"})
    health_summary = DailySummary.model_validate(response)

    if verbose is True:
        print(f"Date of summary: {date_for_summary}")
        print(f"Baseline period: {past_period_start} to {past_period_end}")
        print("=" * 10, "Reasoning", "=" * 10)
        print(reasoning)
        print("=" * 10, "Health Summary", "=" * 10)
        print(health_summary)

    return health_summary

In [ ]:
print(get_daily_summary(garmin, "2025-08-19", "deepseek-chat").model_dump_json())

{"day_type":"balanced","title":"A well-balanced day with improved recovery and lower stress.","emoji":"⚖️","observation":"Resting heart rate decreased by 9.3% to 39, and stress level dropped significantly by 58.8%, indicating excellent recovery. Body battery started high at 82 and ended at 72, showing efficient energy use despite slightly reduced sleep and activity.","recommendation":"Maintain this balance by continuing with moderate exercise and ensuring adequate sleep. Focus on stress management to keep levels low and support ongoing recovery."}


In [ ]:
get_daily_summary(garmin, "2025-08-19", "deepseek-reasoner", verbose=True)

Date of summary: 2025-08-19
Baseline period: 2025-08-12 to 2025-08-18
========== Reasoning ==========
First, I need to generate a JSON summary based on the user's health and fitness data for today, compared to the past 7 days. The JSON must include: day_type, title, emoji, observation, and recommendation.

From the schema, day_type must be one of: 'training', 'active_recovery', 'rest', 'high_stress', 'balanced'.

Looking at the data:

- Resting Heart Rate: 39 (better lower, -9.3% vs baseline, flat trend) – good, lower is better.

- Exercise Minutes: 43 (+16.2% vs baseline, up trend) – higher is better for exercise.

- Stress Level: 12 (-58.8% vs baseline, up trend) – lower is better, so this is great, stress is low.

- Sleep Hours: 6.15 (-6.5% vs baseline, down trend) – lower is not better, so sleep is less than average, not ideal.

- Steps: 7407 (-37.3% vs baseline, up trend) – lower is not better, so steps are down significantly.

- Total Distance Meters: 8155 (-28.8% vs baseline, up

DailySummary(day_type=<DayType.ACTIVE_RECOVERY: 'active_recovery'>, title='Low stress day with excellent body battery recovery despite reduced activity.', emoji='😌', observation='Stress levels dropped significantly by 58.8% to 12, and body battery ended very high at 72, indicating strong recovery. However, sleep was slightly below average at 6.15 hours, and steps decreased by 37.3% compared to baseline.', recommendation='Aim for at least 7 hours of sleep tonight to enhance recovery further. Consider light activities like walking or yoga tomorrow to maintain low stress and support overall well-being.')